In [4]:
import dask_preperation as dp
from dask.distributed import Client, wait
import model as md
import data_preperation as data_prep

In [5]:

dask = dp.Dask(1, 8)
c = Client(dask.cluster)
dask.set_client(c)
c

Client Scheduler: tcp://127.0.0.1:44081 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 540.49 GB


In [6]:
cols = ['FID', 'Date']
pred = 'Depth_m'

data = data_prep.Data('data/pts_merged_final.csv', pred, cols, 42)

 - from DATA: reading csv into GPU memory
 - from DATA: done reading csv into GPU memory
 - from DATA: dropped column: FID
 - from DATA: dropped column: Date


In [7]:
cv_train, cv_test, l_train, l_test = data.split(0.20)

In [8]:
hyperparameters = {	   'N_ESTIMATORS' : 2000,
					   'SPLIT_ALGO' : 1,
					   'SPLIT_CRITERION' : 2,
					   'BOOTSTRAP' : True,
					   'BOOTSTRAP_FEATURES' : False,
					   'ROWS_SAMPLE' : 1.0,
					   'MAX_DEPTH' : 16,
					   'MAX_LEAVES' : -1,
					   'MAX_FEATURES' : 'auto',
					   'N_BINS' : 8,
					   'MIN_ROWS_PER_NODE' : 2,
					   'MIN_IMPURITY_DECREASE' : 0.0,
					   'ACCURACY_METRIC' : 'mean_ae', # 'mse' #'r2' # 'median_aw' # 
					   'QUANTILEPT' : False,
					   'SEED' :  42,
					   'VERBOSE' : False
					   }


In [9]:
print(hyperparameters)

{'N_ESTIMATORS': 2000, 'SPLIT_ALGO': 1, 'SPLIT_CRITERION': 2, 'BOOTSTRAP': True, 'BOOTSTRAP_FEATURES': False, 'ROWS_SAMPLE': 1.0, 'MAX_DEPTH': 16, 'MAX_LEAVES': -1, 'MAX_FEATURES': 'auto', 'N_BINS': 8, 'MIN_ROWS_PER_NODE': 2, 'MIN_IMPURITY_DECREASE': 0.0, 'ACCURACY_METRIC': 'mean_ae', 'QUANTILEPT': False, 'SEED': 42, 'VERBOSE': False}


In [10]:
rf = md.DaskCumlRF(hyperparameters)

In [13]:
cv_dt, l_dt = dask.distribute(cv_train, l_train)
cv_dte, l_dte = dask.distribute(cv_test, l_test)

In [14]:
%%time

rf.train(cv_dt, l_dt)

CPU times: user 217 ms, sys: 54 ms, total: 271 ms
Wall time: 7.44 s


In [15]:
rf.get_metrics(cv_dte, l_test)

Scores ------
 MAE:  1.0146061
  r2:  -0.0685635250369434
 MSE:  3.2579317


(1.0146061, -0.0685635250369434, 3.2579317)

In [19]:
rf.feature_importances(cv_dt, l_dt, True)

AttributeError: 'DaskCumlRF' object has no attribute 'feature_importances'